In [1]:
from google.colab import drive
import nbformat, os

drive.mount('/content/drive', force_remount=True)
SRC = '/content/drive/MyDrive/Colab Notebooks/Untitled13.ipynb'
DST = '/content/drive/MyDrive/colab_saves/RAG for DS and QWEN.ipynb'

with open(SRC, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)


nb.metadata.pop('widgets', None)
for cell in nb.cells:
    cell.metadata.pop('widgets', None)
os.makedirs(os.path.dirname(DST), exist_ok=True)
with open(DST, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

print("✅ Clean copy saved to", DST)

Mounted at /content/drive
✅ Clean copy saved to /content/drive/MyDrive/colab_saves/RAG for DS and QWEN.ipynb


In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

MODEL_ID = "Qwen/Qwen3-Embedding-0.6B"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModel.from_pretrained(
    MODEL_ID,
    output_hidden_states=True,
    torch_dtype=torch.float16,
    device_map="auto")



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

In [ ]:


import pickle, pandas as pd
from google.colab import drive
import os

drive.mount('/content/drive')
with open('/content/drive/My Drive/colab_saves/company_dfs.pkl', 'rb') as f:
    labeled_groups = pickle.load(f)

labeled_group2 = labeled_groups['group2']
labeled_group3 = labeled_groups['group3']
labeled_group4 = labeled_groups['group4']

for name, group in [('group2', labeled_group2),
                    ('group3', labeled_group3),
                    ('group4', labeled_group4)]:
    print(f"\n{name}:")
    for sym, df in group.items():
        print(sym, df.shape, list(df.columns)[:5])



Mounted at /content/drive

group2:
C (400, 21) ['id', 'type', 'publishOn', 'attributes.isLockedPro', 'attributes.commentCount']
GS (400, 21) ['id', 'type', 'publishOn', 'attributes.isLockedPro', 'attributes.commentCount']
MS (400, 21) ['id', 'type', 'publishOn', 'attributes.isLockedPro', 'attributes.commentCount']
USB (400, 21) ['id', 'type', 'publishOn', 'attributes.isLockedPro', 'attributes.commentCount']

group3:
AAPL (400, 21) ['id', 'type', 'publishOn', 'attributes.isLockedPro', 'attributes.commentCount']
MSFT (400, 21) ['id', 'type', 'publishOn', 'attributes.isLockedPro', 'attributes.commentCount']
AMZN (400, 21) ['id', 'type', 'publishOn', 'attributes.isLockedPro', 'attributes.commentCount']
GOOGL (400, 21) ['id', 'type', 'publishOn', 'attributes.isLockedPro', 'attributes.commentCount']

group4:
META (400, 21) ['id', 'type', 'publishOn', 'attributes.isLockedPro', 'attributes.commentCount']
NVDA (400, 21) ['id', 'type', 'publishOn', 'attributes.isLockedPro', 'attributes.commentCo

In [ ]:

import re, html, pandas as pd

def _first_para_from_html(s: str, max_chars=420):
    s = str(s)
    s = re.sub(r"<[^>]+>", " ", s)         # remove tags
    s = html.unescape(" ".join(s.split())) # remove whitespace
    return s[:max_chars]

def make_snippet_df(group_dict: dict[str, pd.DataFrame]) -> pd.DataFrame:
    rows = []
    for tkr, df in group_dict.items():
        # keep needed columns
        need = ["symbol", "publishOn", "title"]
        if "attributes.content" in df.columns:
            need.append("attributes.content") # add the aim body of news if it exist
        sdf = df[need].copy()
        sdf["publishOn"] = pd.to_datetime(sdf["publishOn"], utc=True, errors="coerce") #convert to utc time
        sdf = sdf.dropna(subset=["publishOn"])

        for sym, ts, ttl in sdf[["symbol","publishOn","title"]].itertuples(index=False, name=None):
            rows.append({"ticker": sym, "time": ts, "text": str(ttl), "kind": "headline"})

        if "attributes.content" in sdf.columns:
            mask = sdf["attributes.content"].notna()
            for sym, ts, body in sdf.loc[mask, ["symbol","publishOn","attributes.content"]].itertuples(index=False, name=None):
                rows.append({"ticker": sym, "time": ts,
                             "text": _first_para_from_html(body), "kind": "first_para"})

    snips = pd.DataFrame(rows).drop_duplicates(subset=["ticker","time","text"]).reset_index(drop=True)
    return snips



In [ ]:

snips_df_2 = make_snippet_df(labeled_group2)
print(snips_df_2.head(), snips_df_2.shape)

snips_df_3 = make_snippet_df(labeled_group3)
print(snips_df_3.head(), snips_df_3.shape)


snips_df_4 = make_snippet_df(labeled_group3)
print(snips_df_4.head(), snips_df_3.shape)



  ticker                      time  \
0      C 2025-07-15 20:38:20+00:00   
1      C 2025-07-15 19:49:24+00:00   
2      C 2025-07-15 17:45:07+00:00   
3      C 2025-07-15 13:00:54+00:00   
4      C 2025-07-15 12:51:29+00:00   

                                                text      kind  
0  Citi targets $84B revenue for 2025 while advan...  headline  
1  Citi's June credit card delinquencies, charge-...  headline  
2  Citigroup stock climbs as it ramps up stock bu...  headline  
3  5 stocks to watch on Tuesday: C, JPM, WFC, BLK...  headline  
4  Citigroup boosts NII guidance after Q2 earning...  headline   (2320, 4)
  ticker                      time  \
0   AAPL 2025-07-16 06:07:21+00:00   
1   AAPL 2025-07-16 01:25:36+00:00   
2   AAPL 2025-07-15 19:22:41+00:00   
3   AAPL 2025-07-15 18:07:31+00:00   
4   AAPL 2025-07-15 15:05:04+00:00   

                                                text      kind  
0         Trump eyes drug, chip tariffs by month-end  headline  
1  MP Materi

In [ ]:
import numpy as np

MAX_LEN = 128
# generate embeddings
@torch.no_grad()
def qwen_embed_batched(texts, batch_size=128, pool="mean"):
    if isinstance(texts, str):
        texts = [texts]
    vecs = []
    for i in range(0, len(texts), batch_size):
        chunk = texts[i:i+batch_size]
        enc = tokenizer(
            chunk,
            padding=True,
            truncation=True,
            max_length=MAX_LEN,
            return_tensors="pt"
        ).to(model.device)
        out = model(**enc)
        H = out.hidden_states[-1] if hasattr(out, "hidden_states") else out.last_hidden_state
        E = H.mean(dim=1) if pool == "mean" else H[:, 0]
        E = torch.nn.functional.normalize(E.float(), p=2, dim=1)
        vecs.append(E.cpu().numpy().astype("float32"))
        del enc, out, H, E
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    return np.vstack(vecs)


In [ ]:

class TickerIndex:
    def __init__(self, snips_df: pd.DataFrame, batch_size=128):
        self.df_by_tkr, self.vec_by_tkr = {}, {}
        for tkr, sdf in snips_df.groupby("ticker"):
            texts = sdf["text"].tolist()
            vecs  = qwen_embed_batched(texts, batch_size=batch_size)
            self.df_by_tkr[tkr]  = sdf.reset_index(drop=True)
            self.vec_by_tkr[tkr] = vecs

# given a query, search most relavant snipet by cos similarity
    def retrieve(self, ticker: str, cutoff_time, query_text: str | None = None,
                top_r: int = 2, qvec: np.ndarray | None = None):
        if ticker not in self.df_by_tkr:
            return []
        sdf  = self.df_by_tkr[ticker]
        vecs = self.vec_by_tkr[ticker]
        mask = sdf["time"] <= pd.to_datetime(cutoff_time, utc=True)
        if not mask.any():
            return []
        sub  = sdf[mask]
        subV = vecs[mask.values].astype("float32")
        if qvec is None:
            assert query_text is not None, "Need query_text when qvec is None"
            q = qwen_embed_batched([query_text])[0].astype("float32")
        else:
            q = qvec.astype("float32")

        sims = subV @ q
        top  = sims.argsort()[-top_r:][::-1]
        return sub.iloc[top]["text"].tolist()



In [ ]:

idx_2 = TickerIndex(snips_df_2)
idx_3 = TickerIndex(snips_df_3)
idx_4 = TickerIndex(snips_df_4)


In [ ]:

_title_cache: dict[str, np.ndarray] = {}

def embed_title_cached(text: str) -> np.ndarray:
    k = text.strip()
    v = _title_cache.get(k)
    if v is None:
        v = qwen_embed_batched([k])[0]
        _title_cache[k] = v
    return v



In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

def rag_features_from_group(group_dict, idx: TickerIndex, top_r=1): # concatenate snippets to embeddings

    X, y, meta = [], [], []

    for tkr, df in group_dict.items():
        sdf = df[["symbol","publishOn","title","label"]].copy()
        sdf["publishOn"] = pd.to_datetime(sdf["publishOn"], utc=True, errors="coerce")
        sdf = sdf.dropna(subset=["publishOn"]).reset_index(drop=True)
        titles = sdf["title"].astype(str).tolist()
        title_vecs = qwen_embed_batched(titles, batch_size=16)


        for i, (sym, ts, ttl, lab) in enumerate(sdf.itertuples(index=False, name=None)):
            qvec = title_vecs[i]

            ctx_texts = idx.retrieve(sym, ts, query_text=None, top_r=top_r, qvec=qvec)


            if ctx_texts:
                ctx_vec = qwen_embed_batched(ctx_texts, batch_size=16).mean(axis=0)
            else:
                ctx_vec = np.zeros_like(qvec)

            X.append(np.concatenate([qvec, ctx_vec], axis=0))
            y.append(int(lab))
            meta.append((sym, ts, ttl))

            if i % 200 == 0:
                print(f"[{tkr}] row {i}/{len(sdf)}", flush=True)

    return np.vstack(X), np.asarray(y), meta





In [ ]:
import gc, torch


idx_2 = TickerIndex(snips_df_2, batch_size=16)
X2, y2, meta2 = rag_features_from_group(labeled_group2, idx_2, top_r=1)
del idx_2; gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache(); torch.cuda.ipc_collect()


idx_3 = TickerIndex(snips_df_3, batch_size=16)
X3, y3, meta3 = rag_features_from_group(labeled_group3, idx_3, top_r=1)
del idx_3; gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache(); torch.cuda.ipc_collect()


idx_4 = TickerIndex(snips_df_4, batch_size=16)
X4, y4, meta4 = rag_features_from_group(labeled_group4, idx_4, top_r=1)
del idx_4; gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache(); torch.cuda.ipc_collect()



[C] row 0/400
[C] row 200/400
[GS] row 0/400
[GS] row 200/400
[MS] row 0/400
[MS] row 200/400
[USB] row 0/400
[USB] row 200/400
[AAPL] row 0/400
[AAPL] row 200/400
[MSFT] row 0/400
[MSFT] row 200/400
[AMZN] row 0/400
[AMZN] row 200/400
[GOOGL] row 0/400
[GOOGL] row 200/400
[META] row 0/400
[META] row 200/400
[NVDA] row 0/400
[NVDA] row 200/400
[TSLA] row 0/400
[TSLA] row 200/400
[NFLX] row 0/400
[NFLX] row 200/400


In [ ]:
import gc, torch, numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

results = {}

def make_snippet_df(group_dict: dict[str, pd.DataFrame]) -> pd.DataFrame:
    import re, html
    def _first_para_from_html(s: str, max_chars=420):
        s = str(s)
        s = re.sub(r"<[^>]+>", " ", s)
        s = html.unescape(" ".join(s.split()))
        return s[:max_chars]

    rows = []
    for tkr, df in group_dict.items():
        need = ["symbol", "publishOn", "title"]
        if "attributes.content" in df.columns:
            need.append("attributes.content")
        sdf = df[need].copy()
        sdf["publishOn"] = pd.to_datetime(sdf["publishOn"], utc=True, errors="coerce")
        sdf = sdf.dropna(subset=["publishOn"])
        for sym, ts, ttl in sdf[["symbol","publishOn","title"]].itertuples(index=False, name=None):
            rows.append({"ticker": sym, "time": ts, "text": str(ttl), "kind": "headline"})
        if "attributes.content" in sdf.columns:
            mask = sdf["attributes.content"].notna()
            for sym, ts, body in sdf.loc[mask, ["symbol","publishOn","attributes.content"]].itertuples(index=False, name=None):
                rows.append({"ticker": sym, "time": ts,
                             "text": _first_para_from_html(body), "kind": "first_para"})
    snips = pd.DataFrame(rows).drop_duplicates(subset=["ticker","time","text"]).reset_index(drop=True)
    return snips


for group_name, labeled_group in [
    ("group2", labeled_group2),
    ("group3", labeled_group3),
    ("group4", labeled_group4),
]:
    print(f"\n=== Tuning on {group_name} ===")

    snips_df = make_snippet_df(labeled_group)
    idx = TickerIndex(snips_df, batch_size=16)

    X, y, meta = rag_features_from_group(labeled_group, idx, top_r=1)


    del idx; gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache(); torch.cuda.ipc_collect()


    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )


    emb_train = torch.from_numpy(X_train).to(device).float()
    emb_test  = torch.from_numpy(X_test ).to(device).float()
    y_train_t = torch.from_numpy(y_train).to(device).long()


    d = emb_train.size(1)
    head = nn.Sequential(
        nn.Linear(d, 64),
        nn.ReLU(),
        nn.Linear(64, 2)
    ).to(device)


    optimizer = torch.optim.Adam(head.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()


    train_ds = TensorDataset(emb_train, y_train_t)
    train_dl = DataLoader(train_ds, batch_size=16, shuffle=True)


    best_prev_loss = float("inf")
    tol = 1e-4
    patience = 3
    stale_epochs = 0
    max_epochs = 100

    for epoch in range(1, max_epochs + 1):
        head.train()
        epoch_loss = 0.0
        for emb_b, lab_b in train_dl:
            optimizer.zero_grad()
            logits = head(emb_b)
            loss = criterion(logits, lab_b)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * emb_b.size(0)

        avg_loss = epoch_loss / len(train_ds)
        print(f"  Epoch {epoch:>3} — Loss: {avg_loss:.6f}")
        improvement = best_prev_loss - avg_loss
        if improvement < tol:
            stale_epochs += 1
            print(f"  ↳ small improvement (Δ={improvement:.2e}) stale {stale_epochs}/{patience}")
        else:
            stale_epochs = 0
            best_prev_loss = avg_loss
        if stale_epochs >= patience:
            print(f"  ↳ Early stopping at epoch {epoch}")
            break


    head.eval()
    with torch.no_grad():
        preds = head(emb_test).argmax(dim=1).detach().cpu().numpy()

    acc = accuracy_score(y_test, preds)
    micro_f1 = f1_score(y_test, preds, average="micro")
    macro_f1 = f1_score(y_test, preds, average="macro")
    weighted_f1 = f1_score(y_test, preds, average="weighted")
    macro_recall = recall_score(y_test, preds, average="macro")

    print(f"Metrics for {group_name}:")
    print(f"  Accuracy    : {acc*100:6.2f}%")
    print(f"  Micro-F1    : {micro_f1*100:6.2f}%")
    print(f"  Macro-F1    : {macro_f1*100:6.2f}%")
    print(f"  Weighted-F1 : {weighted_f1*100:6.2f}%")
    print(f"  Macro-Recall: {macro_recall*100:6.2f}%\n")

    results[group_name] = head


    del emb_train, emb_test, y_train_t, head
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache(); torch.cuda.ipc_collect()



=== Tuning on group2 ===
[C] row 0/400
[C] row 200/400
[GS] row 0/400
[GS] row 200/400
[MS] row 0/400
[MS] row 200/400
[USB] row 0/400
[USB] row 200/400
  Epoch   1 — Loss: 0.690810
  Epoch   2 — Loss: 0.687239
  Epoch   3 — Loss: 0.684674
  Epoch   4 — Loss: 0.682368
  Epoch   5 — Loss: 0.678872
  Epoch   6 — Loss: 0.676158
  Epoch   7 — Loss: 0.672376
  Epoch   8 — Loss: 0.669481
  Epoch   9 — Loss: 0.666599
  Epoch  10 — Loss: 0.664147
  Epoch  11 — Loss: 0.659586
  Epoch  12 — Loss: 0.656305
  Epoch  13 — Loss: 0.652638
  Epoch  14 — Loss: 0.649676
  Epoch  15 — Loss: 0.645413
  Epoch  16 — Loss: 0.642419
  Epoch  17 — Loss: 0.639104
  Epoch  18 — Loss: 0.636601
  Epoch  19 — Loss: 0.634641
  Epoch  20 — Loss: 0.629648
  Epoch  21 — Loss: 0.627287
  Epoch  22 — Loss: 0.623647
  Epoch  23 — Loss: 0.621115
  Epoch  24 — Loss: 0.618727
  Epoch  25 — Loss: 0.617229
  Epoch  26 — Loss: 0.614277
  Epoch  27 — Loss: 0.609355
  Epoch  28 — Loss: 0.609942
  ↳ small improvement (Δ=-5.86e-04

In [ ]:

from transformers import AutoTokenizer, AutoModel
import torch, numpy as np

def build_embedder(model_id, max_len=256, dtype=torch.float16):
    tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    mdl = AutoModel.from_pretrained(
        model_id,
        output_hidden_states=False,
        torch_dtype=dtype,
        device_map="auto",
        trust_remote_code=True,
    )

    @torch.no_grad()
    def embed_batched(texts, batch_size=16, pool="mean"):
        if isinstance(texts, str):
            texts = [texts]
        vecs = []
        for i in range(0, len(texts), batch_size):
            enc = tok(
                texts[i:i+batch_size],
                padding=True,
                truncation=True,
                max_length=max_len,
                return_tensors="pt"
            ).to(mdl.device)
            out = mdl(**enc).last_hidden_state
            if pool == "mean":
                E = out.mean(dim=1)
            else:
                E = out[:, 0]   # CLS token
            E = torch.nn.functional.normalize(E, p=2, dim=1)
            vecs.append(E.cpu().numpy().astype("float32"))
        return np.vstack(vecs)

    return tok, mdl, embed_batched


In [ ]:
MODEL_ID = "deepseek-ai/deepseek-llm-7b-base"
tokenizer, model, deepseek_embed_batched = build_embedder(MODEL_ID, max_len=256)

qwen_embed_batched = deepseek_embed_batched

for group_name, labeled_group in [
    ("group2", labeled_group2),
    ("group3", labeled_group3),
    ("group4", labeled_group4),
]:
    print(f"\n=== Tuning on {group_name} ===")

    snips_df = make_snippet_df(labeled_group)
    idx = TickerIndex(snips_df, batch_size=16)

    X, y, meta = rag_features_from_group(labeled_group, idx, top_r=1)


    del idx; gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache(); torch.cuda.ipc_collect()


    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )


    emb_train = torch.from_numpy(X_train).to(device).float()
    emb_test  = torch.from_numpy(X_test ).to(device).float()
    y_train_t = torch.from_numpy(y_train).to(device).long()


    d = emb_train.size(1)
    head = nn.Sequential(
        nn.Linear(d, 64),
        nn.ReLU(),
        nn.Linear(64, 2)
    ).to(device)


    optimizer = torch.optim.Adam(head.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()


    train_ds = TensorDataset(emb_train, y_train_t)
    train_dl = DataLoader(train_ds, batch_size=16, shuffle=True)


    best_prev_loss = float("inf")
    tol = 1e-4
    patience = 3
    stale_epochs = 0
    max_epochs = 100

    for epoch in range(1, max_epochs + 1):
        head.train()
        epoch_loss = 0.0
        for emb_b, lab_b in train_dl:
            optimizer.zero_grad()
            logits = head(emb_b)
            loss = criterion(logits, lab_b)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * emb_b.size(0)

        avg_loss = epoch_loss / len(train_ds)
        print(f"  Epoch {epoch:>3} — Loss: {avg_loss:.6f}")
        improvement = best_prev_loss - avg_loss
        if improvement < tol:
            stale_epochs += 1
            print(f"  ↳ small improvement (Δ={improvement:.2e}) stale {stale_epochs}/{patience}")
        else:
            stale_epochs = 0
            best_prev_loss = avg_loss
        if stale_epochs >= patience:
            print(f"  ↳ Early stopping at epoch {epoch}")
            break


    head.eval()
    with torch.no_grad():
        preds = head(emb_test).argmax(dim=1).detach().cpu().numpy()

    acc = accuracy_score(y_test, preds)
    micro_f1 = f1_score(y_test, preds, average="micro")
    macro_f1 = f1_score(y_test, preds, average="macro")
    weighted_f1 = f1_score(y_test, preds, average="weighted")
    macro_recall = recall_score(y_test, preds, average="macro")

    print(f"Metrics for {group_name}:")
    print(f"  Accuracy    : {acc*100:6.2f}%")
    print(f"  Micro-F1    : {micro_f1*100:6.2f}%")
    print(f"  Macro-F1    : {macro_f1*100:6.2f}%")
    print(f"  Weighted-F1 : {weighted_f1*100:6.2f}%")
    print(f"  Macro-Recall: {macro_recall*100:6.2f}%\n")

    results[group_name] = head


    del emb_train, emb_test, y_train_t, head
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache(); torch.cuda.ipc_collect()

tokenizer_config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Tuning on group2 ===
[C] row 0/400
[C] row 200/400
[GS] row 0/400
[GS] row 200/400
[MS] row 0/400
[MS] row 200/400
[USB] row 0/400
[USB] row 200/400
  Epoch   1 — Loss: 0.690739
  Epoch   2 — Loss: 0.689386
  Epoch   3 — Loss: 0.688964
  Epoch   4 — Loss: 0.688339
  Epoch   5 — Loss: 0.687413
  Epoch   6 — Loss: 0.686865
  Epoch   7 — Loss: 0.686223
  Epoch   8 — Loss: 0.685566
  Epoch   9 — Loss: 0.684749
  Epoch  10 — Loss: 0.683760
  Epoch  11 — Loss: 0.682778
  Epoch  12 — Loss: 0.681953
  Epoch  13 — Loss: 0.680812
  Epoch  14 — Loss: 0.680055
  Epoch  15 — Loss: 0.679196
  Epoch  16 — Loss: 0.678070
  Epoch  17 — Loss: 0.677233
  Epoch  18 — Loss: 0.675716
  Epoch  19 — Loss: 0.674329
  Epoch  20 — Loss: 0.673710
  Epoch  21 — Loss: 0.672379
  Epoch  22 — Loss: 0.671795
  Epoch  23 — Loss: 0.670310
  Epoch  24 — Loss: 0.669118
  Epoch  25 — Loss: 0.668651
  Epoch  26 — Loss: 0.666446
  Epoch  27 — Loss: 0.665496
  Epoch  28 — Loss: 0.663270
  Epoch  29 — Loss: 0.663179
  ↳ s

In [ ]:
# do embedding without rag
def title_only_features_from_group(group_dict, batch_size=16):

    X, y, meta = [], [], []
    for tkr, df in group_dict.items():
        sdf = df[["symbol", "publishOn", "title", "label"]].copy()
        sdf["publishOn"] = pd.to_datetime(sdf["publishOn"], utc=True, errors="coerce")
        sdf = sdf.dropna(subset=["publishOn"]).reset_index(drop=True)

        titles = sdf["title"].astype(str).tolist()
        title_vecs = qwen_embed_batched(titles, batch_size=batch_size)

        X.append(title_vecs)
        y.extend(sdf["label"].astype(int).tolist())
        meta.extend(list(sdf[["symbol","publishOn","title"]].itertuples(index=False, name=None)))

    return np.vstack(X), np.asarray(y), meta


In [ ]:
import gc, torch, numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_ID = "deepseek-ai/deepseek-llm-7b-base"
tokenizer, model, deepseek_embed_batched = build_embedder(MODEL_ID, max_len=256)
qwen_embed_batched = deepseek_embed_batched

results = {}
for group_name, labeled_group in [
    ("group2", labeled_group2),
    ("group3", labeled_group3),
    ("group4", labeled_group4),
]:
    print(f"\n=== Tuning on {group_name} (DeepSeek title-only) ===")

    X, y, meta = title_only_features_from_group(labeled_group, batch_size=16)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    emb_train = torch.from_numpy(X_train).to(device).float()
    emb_test  = torch.from_numpy(X_test ).to(device).float()
    y_train_t = torch.from_numpy(y_train).to(device).long()

    d = emb_train.size(1)
    head = nn.Sequential(
        nn.Linear(d, 64),
        nn.ReLU(),
        nn.Linear(64, 2)
    ).to(device)

    optimizer = torch.optim.Adam(head.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    train_ds = TensorDataset(emb_train, y_train_t)
    train_dl = DataLoader(train_ds, batch_size=16, shuffle=True)

    best_prev_loss = float("inf")
    tol = 1e-4
    patience = 3
    stale_epochs = 0
    max_epochs = 100

    for epoch in range(1, max_epochs + 1):
        head.train()
        epoch_loss = 0.0
        for emb_b, lab_b in train_dl:
            optimizer.zero_grad()
            logits = head(emb_b)
            loss = criterion(logits, lab_b)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * emb_b.size(0)

        avg_loss = epoch_loss / len(train_ds)
        print(f"  Epoch {epoch:>3} — Loss: {avg_loss:.6f}")
        improvement = best_prev_loss - avg_loss
        if improvement < tol:
            stale_epochs += 1
            print(f"  ↳ small improvement (Δ={improvement:.2e}) stale {stale_epochs}/{patience}")
        else:
            stale_epochs = 0
            best_prev_loss = avg_loss
        if stale_epochs >= patience:
            print(f"  ↳ Early stopping at epoch {epoch}")
            break

    head.eval()
    with torch.no_grad():
        preds = head(emb_test).argmax(dim=1).detach().cpu().numpy()

    acc = accuracy_score(y_test, preds)
    micro_f1 = f1_score(y_test, preds, average="micro")
    macro_f1 = f1_score(y_test, preds, average="macro")
    weighted_f1 = f1_score(y_test, preds, average="weighted")
    macro_recall = recall_score(y_test, preds, average="macro")

    print(f"Metrics for {group_name}:")
    print(f"  Accuracy    : {acc*100:6.2f}%")
    print(f"  Micro-F1    : {micro_f1*100:6.2f}%")
    print(f"  Macro-F1    : {macro_f1*100:6.2f}%")
    print(f"  Weighted-F1 : {weighted_f1*100:6.2f}%")
    print(f"  Macro-Recall: {macro_recall*100:6.2f}%\n")

    results[group_name] = head

    del emb_train, emb_test, y_train_t, head
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache(); torch.cuda.ipc_collect()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Tuning on group2 (DeepSeek title-only) ===
  Epoch   1 — Loss: 0.691579
  Epoch   2 — Loss: 0.690042
  Epoch   3 — Loss: 0.689540
  Epoch   4 — Loss: 0.689319
  Epoch   5 — Loss: 0.688854
  Epoch   6 — Loss: 0.688698
  Epoch   7 — Loss: 0.688381
  Epoch   8 — Loss: 0.687995
  Epoch   9 — Loss: 0.687813
  Epoch  10 — Loss: 0.687371
  Epoch  11 — Loss: 0.686933
  Epoch  12 — Loss: 0.686604
  Epoch  13 — Loss: 0.686286
  Epoch  14 — Loss: 0.686043
  Epoch  15 — Loss: 0.685431
  Epoch  16 — Loss: 0.685193
  Epoch  17 — Loss: 0.684723
  Epoch  18 — Loss: 0.684401
  Epoch  19 — Loss: 0.683933
  Epoch  20 — Loss: 0.683255
  Epoch  21 — Loss: 0.683031
  Epoch  22 — Loss: 0.682438
  Epoch  23 — Loss: 0.681942
  Epoch  24 — Loss: 0.681587
  Epoch  25 — Loss: 0.681243
  Epoch  26 — Loss: 0.680888
  Epoch  27 — Loss: 0.680315
  Epoch  28 — Loss: 0.679765
  Epoch  29 — Loss: 0.678868
  Epoch  30 — Loss: 0.678604
  Epoch  31 — Loss: 0.677926
  Epoch  32 — Loss: 0.677327
  Epoch  33 — Loss: 0.67